In [1]:

import numpy as np
import pandas as pd
import json
import os

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm.autonotebook import tqdm

C:\Users\gabeb\AppData\Local\Temp\ipykernel_27348\2226318540.py:15: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
# CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"
chromedriver_path = 'C:/Users/gabeb/Desktop/programming/chromedriver-win64/chromedriver.exe'

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driver
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--start-fullscreen",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            "user-agent=Chrome/116.0.5845.96"
        ],
    }
    options = add_driver_options(driver_config["options"])
    # options.binary_location = CHROME_BINARY_LOCATION
    driver = webdriver.Chrome(
        executable_path=chromedriver_path,
        options=options)
    return driver

In [3]:
def parse_rollercoaster_page(soup):
    # Initialize variables to store information
    city = region = country = make = None
    trackStats = {}
    image_urls = []

    # Find location information
    feature = soup.find('div', {"id": "feature"})
    if feature:
        descList = feature.findAll('div')
        if descList:
            desc = descList[0]
            locationLinks = desc.findAll('a')
            if len(locationLinks) == 3: # amusement park, city, country
                city = locationLinks[1].contents[0]
                region = locationLinks[1].contents[0]  # Assuming region is the same as city
                country = locationLinks[2].contents[0]
            else:
                city = locationLinks[-3].contents[0]
                region = locationLinks[-2].contents[0]
                country = locationLinks[-1].contents[0]

            # Find make information
            if len(descList) >= 2:
                make_link = descList[1].find('a')
                if make_link:
                    make = make_link.contents[0]

    # Find track information
    trackStatsTable = soup.find('table', {'class': 'stat-tbl'})
    if trackStatsTable:
        trackStatsTag = trackStatsTable.findAll('tr')
        for tr in trackStatsTag:
            key = tr.find('th').text.strip()
            valueTag = tr.find('td')
            if valueTag:
                value = valueTag.text.strip()
                if key == "Elements": # list of links
                    value = [a.contents[0] for a in valueTag.findAll('a')]
                elif valueTag.find('span', {'class': 'float'}) is not None: # float value
                    unit = valueTag.contents[1].strip() if len(valueTag.contents) >= 2 else ""
                    key = f"{key} ({unit})"
                    value = valueTag.find('span', {'class': 'float'}).text.strip()
            else:
                value = None
            trackStats[key] = value

    # Find images
    image_tags = soup.find_all('img', {'class': 'image'})
    for img in image_tags:
        image_url = urljoin(rcdb_url, img['src'])
        image_urls.append(image_url)

    return city, region, country, make, trackStats, image_urls

def parse_rollercoaster_table_row(row):
    # Get data from the table row
    elements = row.findAll('td')
    coasterName = elements[1].find('a').text.strip()
    coasterLink = elements[1].find('a')['href']
    amusementPark = elements[2].find('a').text.strip()
    coasterType = elements[3].find('a').text.strip()
    coasterDesign = elements[4].find('a').text.strip()
    statusTag = elements[5].find('a')
    status = statusTag.contents[0] if statusTag else "Removed"
    openedTag = elements[6].find('time')
    opened = openedTag.contents[0] if openedTag else "unknown"

    # Get data from the coaster specific link
    driver.get(f"{rcdb_url}{coasterLink}")
    coaster_soup = BeautifulSoup(driver.page_source, 'html')
    city, region, country, make, trackStats, image_urls = parse_rollercoaster_page(coaster_soup)

    return {
        'Name': coasterName, 
        'Amusement Park': amusementPark, 
        'Type': coasterType, 
        'Design': coasterDesign, 
        'Status': status, 
        'Opened': opened, 
        'City': city, 
        'Region': region, 
        'Country': country, 
        'Make': make,
        **trackStats,
        'Image URLs': image_urls
    }


In [4]:
rcdb_url = "https://rcdb.com"

# Path to your ChromeDriver executable
chromedriver_path = r'C:\Users\gabeb\Desktop\programming\chromedriver-win64\chromedriver.exe'

# Initialize Chrome WebDriver
# service = Service(chromedriver_path)
# service.start()
driver = webdriver.Chrome()
# driver = initialize_driver()
driver.get(f"{rcdb_url}/r.htm?ot=2")
soup = BeautifulSoup(driver.page_source, 'html')
numPages = int(soup.find('div', {"id": "rfoot"}).findAll('a')[-2].contents[0])
print(f"Total number of pages to iterate through: {numPages}")

pbar = tqdm(range(numPages))
rollercoasters = []

for pageNum in pbar:
    print(f"Getting information for page {pageNum+1}")
    driver.get(f"{rcdb_url}/r.htm?page={pageNum+1}&ot=2")
    soup = BeautifulSoup(driver.page_source, 'html')
    table = soup.findAll('table')[1].find('tbody')
    rows = table.findAll('tr')
    
    for row in rows:
        rowData = parse_rollercoaster_table_row(row)
        pbar.set_postfix(
            coasterName=rowData['Name'], 
            amusementPark=rowData['Amusement Park'])
        rollercoasters.append(rowData)
        
    # every page, save the information to a dataframe
    df = pd.DataFrame.from_records(rollercoasters)
    df.to_csv("data.csv", index=False)

Total number of pages to iterate through: 496


  0%|          | 0/496 [00:00<?, ?it/s]

Getting information for page 1


  0%|          | 1/496 [00:03<26:26,  3.21s/it, amusementPark=Parque de Atracciones de Madrid, coasterName=Abismo]            

Getting information for page 2


...

100%|█████████▉| 494/496 [26:36<00:06,  3.00s/it, amusementPark=Fenghu Park, coasterName=unknown]                     

Getting information for page 495


100%|█████████▉| 495/496 [26:39<00:03,  3.02s/it, amusementPark=Hongshan Happy Valley, coasterName=unknown]                      

Getting information for page 496


100%|██████████| 496/496 [26:41<00:00,  3.23s/it, amusementPark=Vorontsovskiy Park, coasterName=unknown]                     


In [5]:
driver.quit()

In [16]:
df.head()

,Name,Amusement Park,Type,Design,Status,Opened,City,Region,Country,Make,...,Duration,G-Force (),Vertical Angle (°),Uphill Length (ft),Downhill Length (ft),Δ Elevation (ft),Airtime Points,Crossings,Bank Angle (°),Drop
0,€uro-Coaster,Wiener Prater,Steel,Suspended,Removed,5/29/2020,Vienna,Vienna,Austria,Reverchon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,€uro Coaster,Funland Theme Park,Steel,Sit Down,Removed,2021,Somerset,England,United Kingdom,SBF Visa Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1066,Festyland,Steel,Sit Down,Operating,3/27/2005,Bretteville-Sur-Odon,Normandy,France,Soquet,...,1:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10 Inversion Roller Coaster,Chimelong Paradise,Steel,Sit Down,Operating,2/2006,Guangzhou,Guangdong,China,Intamin Amusement Rides,...,1:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970 Galaxy Rip Tide Coaster,Swampy Jack's Wongo Adventure,Steel,Sit Down,Relocated,6/27/2014,Panama City Beach,Florida,United States,S.D.C.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = df[df['Name'] != 'unknown']


In [25]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox, ttk
from PIL import ImageTk, Image
import requests
from io import BytesIO

def search_and_display_rollercoasters(df):
    results = pd.DataFrame()  # Define results globally

    # Function to filter rollercoasters based on user input
    def filter_rollercoasters(query):
        nonlocal results  # Use nonlocal to modify the results DataFrame
        results = df[df.apply(lambda row: any(query.lower() in str(cell).lower() for cell in row), axis=1)]
        return results

    def display_rollercoasters(rollercoasters):
        rollercoasters_list.delete(0, tk.END)
        for i, (_, rollercoaster) in enumerate(rollercoasters.iterrows(), start=1):
            rollercoasters_list.insert(tk.END, f"{i}. {rollercoaster['Name']} - {rollercoaster['Amusement Park']}")

    def search():
        query = query_entry.get().strip()
        global results  # Declare results as global
        results = filter_rollercoasters(query)
        if results.empty:
            messagebox.showinfo("Info", "No rollercoasters found matching the search query.")
        else:
            display_rollercoasters(results)
            display_rollercoaster_info()

    def display_rollercoaster_info(event=None):
        selection = rollercoasters_list.curselection()
        if selection:
            selected_index = int(selection[0])
            if selected_index < len(results):
                selected_coaster = results.iloc[selected_index]
                rollercoaster_info_text.delete(1.0, tk.END)
                for key, value in selected_coaster.items():
                    try:
                        if pd.notna(value):
                            rollercoaster_info_text.insert(tk.END, f"{key}: {value}\n")
                    except TypeError:
                        pass

    def display_rollercoaster_image(coaster):
        img_url = coaster.get('Image_URL')
        if img_url:
            response = requests.get(img_url)
            if response.status_code == 200:
                img_data = response.content
                img = Image.open(BytesIO(img_data))
                img = img.resize((200, 200), Image.ANTIALIAS)
                rollercoaster_image.image = ImageTk.PhotoImage(img)
                rollercoaster_image.config(image=rollercoaster_image.image)
        else:
            rollercoaster_image.config(image="")  # Clear image if URL is not available

    # Initialize Tkinter window
    window = tk.Tk()
    window.title("Rollercoaster Search")

    # Styling
    window.configure(background="white")
    window.geometry("600x500")

    # Search query entry
    query_label = tk.Label(window, text="Enter search query:", bg="white")
    query_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
    query_entry = tk.Entry(window)
    query_entry.grid(row=0, column=1, padx=5, pady=5)

    # Search button
    search_button = tk.Button(window, text="Search", command=search)
    search_button.grid(row=0, column=2, padx=5, pady=5)

    # Rollercoasters listbox
    rollercoasters_frame = tk.LabelFrame(window, text="Rollercoasters", bg="white")
    rollercoasters_frame.grid(row=1, column=0, columnspan=3, padx=5, pady=5, sticky="we")
    rollercoasters_list = tk.Listbox(rollercoasters_frame, width=50, height=10, bg="white", borderwidth=0)
    rollercoasters_list.grid(row=0, column=0, padx=5, pady=5, sticky="ns")
    rollercoasters_list.bind("<<ListboxSelect>>", display_rollercoaster_info)
    scrollbar = tk.Scrollbar(rollercoasters_frame, orient="vertical", command=rollercoasters_list.yview)
    scrollbar.grid(row=0, column=1, sticky="ns")
    rollercoasters_list.config(yscrollcommand=scrollbar.set)

    # Rollercoaster information text
    info_frame = tk.LabelFrame(window, text="Rollercoaster Information", bg="white")
    info_frame.grid(row=2, column=0, columnspan=3, padx=5, pady=5, sticky="we")
    rollercoaster_info_text = tk.Text(info_frame, width=50, height=10, bg="white", wrap="word", borderwidth=0)
    rollercoaster_info_text.grid(row=0, column=0, padx=5, pady=5, sticky="w")
    scrollbar = tk.Scrollbar(info_frame, orient="vertical", command=rollercoaster_info_text.yview)
    scrollbar.grid(row=0, column=1, sticky="ns")
    rollercoaster_info_text.config(yscrollcommand=scrollbar.set)

    # Rollercoaster image display
    rollercoaster_image = tk.Label(window, bg="white")
    rollercoaster_image.grid(row=3, column=0, columnspan=3, padx=5, pady=5)

    # Display window
    window.mainloop()

# Example usage:
# Assuming df is your Pandas DataFrame containing rollercoaster information
# Call the function passing the DataFrame as argument
search_and_display_rollercoasters(df)


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\gabeb\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\gabeb\AppData\Local\Temp\ipykernel_27348\4058786717.py", line 41, in display_rollercoaster_info
    if pd.notna(value):
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
C:\Users\gabeb\AppData\Local\Temp\ipykernel_27348\4058786717.py:41: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pd.notna(value):
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\gabeb\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users